In [3]:
import tensorflow as tf
from tensorflow import keras
from keras import utils
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPool2D, Input, Dense, Flatten, Concatenate
from keras.callbacks import ModelCheckpoint

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
from IPython.display import Image

import os
import random
from copy import deepcopy
import pickle

In [4]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

In [5]:
fashion_mnist = keras.datasets.fashion_mnist
((x_train, y_train), (x_test, y_test)) = fashion_mnist.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60000, 28, 28)
(60000, 10)
(10000, 28, 28)
(10000, 10)


In [13]:
IMG_SHAPE = (32, 32, 3)
resnet50 = tf.keras.applications.ResNet50(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
len(resnet50.layers)


175

In [8]:
resnet50.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 38, 38, 3)    0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 16, 16, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 16, 16, 64)   256         ['conv1_conv[0][0]']             
                                                                                           

In [22]:
from keras.utils.vis_utils import plot_model
# plot model architecture
plot_model(resnet50, show_shapes=True, to_file='resnet50.png')

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [16]:
resnet50.trainable = True		# resnet 모델 학습동결을 해제한다
# for idx, i in enumerate(resnet50.layers):	# 143층부터의 학습은 해제상태로 두고, 
#   i.trainable = bool_arr[idx]				# 이전까지의 학습은 동결한다.

idx = 0 
for i in resnet50.layers[:]:	# 동결이 제대로 해제됐는지 약간 이전층부터 출력해본다.
  if i.name[:12] == 'conv2_block1':
    print(idx, 'layer :  ', i.name)
  elif i.name[:12] == 'conv2_block2':
    print(idx, 'layer :  ', i.name)
  elif i.name[:12] == 'conv2_block3':
    print(idx, 'layer :  ', i.name)
  elif i.name[:12] == 'conv3_block1':
    print(idx, 'layer :  ', i.name)

    
  idx += 1
  
'''
conv2_block1
conv2_block2
conv2_block3
conv3_block1
conv3_block2
conv3_block3
conv3_block4
conv4_block1
conv4_block2
conv4_block3
conv4_block4
conv4_block5
conv4_block6
conv5_block1
conv5_block2
conv5_block3
'''

7 layer :   conv2_block1_1_conv
8 layer :   conv2_block1_1_bn
9 layer :   conv2_block1_1_relu
10 layer :   conv2_block1_2_conv
11 layer :   conv2_block1_2_bn
12 layer :   conv2_block1_2_relu
13 layer :   conv2_block1_0_conv
14 layer :   conv2_block1_3_conv
15 layer :   conv2_block1_0_bn
16 layer :   conv2_block1_3_bn
17 layer :   conv2_block1_add
18 layer :   conv2_block1_out


In [8]:
IMG_SHAPE = (32, 32, 3)
base_model = MobileNet(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
len(base_model.layers)

86

In [9]:
IMG_SHAPE = (32, 32, 3)
base_model = MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
len(base_model.layers)

9420800/9406464 [==============================] - 2s 0us/step


154

In [10]:
IMG_SHAPE = (32, 32, 3)
base_model = VGG16(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
len(base_model.layers)

19

In [9]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0     

In [10]:
len(base_model.layers)

19

In [11]:
np.random.seed(0)

sample_arr = [True, False]
bool_arr = np.random.choice(sample_arr, size=len(base_model.layers))
print('Numpy Array: ')
print(bool_arr)
print(len(bool_arr))

Numpy Array: 
[ True False False  True False False False False False False False  True
  True False  True  True  True  True  True]
19
